# Control óptimo para un modelo SIR determinista

## Francisco Valentín Valerio López
## Facultad de Ciencias Físico Matemáticas - Benemérita Universidad Autónoma de Puebla

### Introducción

Supongamos que tenemos un escenario en el que una enfermedad infecciosa se propaga entre los individuos de una población de tamaño $N$, los cuales se encuentran distribuidos en distintas regiones geográficas. Cada una de las regiones contiene una subpoblación de tamaño $N_i$, y son tales que $sum_{i=1}^N N_i = N$. Los individuos en cada región pueden desplazarse entre las regiones e interactuar entre sí.

<div style="text-align: center">
    <figure>
        <img src='img\map.jpg' width='600' height='400' />
        <figcaption> Esquema que representa el flujo de habitantes en una población espacialmente estructurada.</figcaption>
    </figure>
</div>

La interacción entre los individuos contagiados y los individuos susceptibles a contagiarse provoca que la enfermedad se propague dentro de la población y se produzca una epidemia. Es por este motivo que, con el objetivo de reducir el impacto de una epidemia en la población, las autoridades de salud pública realizan intervenciones que tienen como objetivo *controlar* la propagación de la enfermedad. Algunas de las estrategias más comunes de intervención son la vacunación, el aislamiento de individuos infectados, medidas de distanciamiento social y el rastreo de contactos.

Sin embargo, la implementación de estas estrategias conllevan un costo económico y social, por lo que es deseable que se logren aplicar de manera óptima al reducir el costo. 

<div style="text-align: center">
    <figure>
        <img src='img\control.jpg' width='500' height='400' style="padding-right: 10px;" />
        <img src='img\cost.jpg' width='500' height='400' style="padding-left: 10px;" />
        <figcaption>Implementar una estrategia de control para reducir el impacto de una epidemia es costoso. Lo ideal es lograr reducir el contagio con la menor cantidad de recursos. </figcaption>
    </figure>
</div>

¿Cómo se puede encontrar la estrategia de control óptima que minimíce el costo de reducir la propagación de la enfermedad? Esta pregunta corresponde a un problema de control óptimo que buscamos resolver con este trabajo.

### Modelo epidemiológico SIR

El [modelo epidemiológico SIR](https://mathworld.wolfram.com/Kermack-McKendrickModel.html) es un modelo matemático que se utiliza para describir la dinámica de una enfermedad infecciosa en una población. Este modelo considera que existen tres grupos, o compartimentos, en los que se divide a la población:

- Susceptibles ($S$): Son los individuos propensos a contagiarse de la enfermedad, es decir, aquellos que no han sido infectados aún y que no son inmunes a la enfermedad.

- Infectados ($I$): Son los individuos que han sido infectados con la enfermedad y son capaces de transmitirla a los susceptibles.

- Recuperados ($R$): Son aquellos que se han recuperado de la enfermedad y que ahora son inmunes a ella.

La transición entre los distintos grupos se entiende mejor utilizando el diagrama de bloques que se muestra a continuación. 

<div style="text-align: center">
    <figure>
        <img src='img\sir_block.png' width='800' height='250' />
        <figcaption> Diagrama de bloques para el modelo SIR.</figcaption>
    </figure>
</div>

Inicialmente, en el compartimento de susceptibles se encuentra la mayor parte de la población, excepto por los individuos que se han infectado de la enfermedad. Posteriormente, los individuos se mueven de $S$ a $I$ a una tasa proporcional al número de contactos entre los susceptibles y los infectados, $\beta SI$. Finalmente, los individuos infectados se mueven de $I$ a $R$ a una tasa proporcional al número actual de infectados, $\gamma I$.

El modelo SIR supone que la población es *cerrada* (sin nacimientos y sin decesos) y que está *mezclada homogéneamente* (es decir, que todos los individuos tienen la misma probabilidad de tener contacto con cualquier otro). Denotando a $S(t), I(t)$ y $R(t)$ como las fracciones de individuos susceptibles, infectados y recuperados al tiempo $t$, respectivamente, la dinámica se expresa a través del siguiente sistema de ecuaciones diferenciales:

$$
    \begin{split}
        \frac{dS}{dt} &= - \beta S I \\
        \frac{dI}{dt} &= \beta S I  - \gamma I \\
        \frac{dR}{dt} &= \gamma I \\
        \end{split} \tag{1}
$$

con las condiciones iniciales: $S(0) = S_0 > 0 , \quad I(0) = I_0 > 0 , \quad R(0)=0$.

Los parámetros $\beta$ y $\gamma$ del modelo (1) son esenciales para describir la dinámica de la enfermedad en la población:

- $\beta$ se conoce como la *tasa de transmisión* y describe cuántas personas susceptibles, en promedio, un individuo infectado contagiará por unidad de tiempo.

- $\gamma$ es la *tasa de recuperación* y representa la fracción de individuos que se recupera y se vuelve inmune a la enfermedad por unidad de tiempo.

Ambos parámetros son esenciales para determinar el *número básico de reproducción*, $\mathcal{R}_0$, que determina el número promedio de personas que un individuo infectado contagiará durante su periodo infeccioso, éste se define como:

$$
    \mathcal{R}_0 \equiv \frac{\beta}{\gamma} \tag{2}
$$

La importancia de $\mathcal{R}_0$ es que nos otorga información acerca del potencial de propagacióon de una enfermedad.

- Si $\mathcal{R}_0 > 1$, cada persona infectada transmitirá la enfermedad a más de una persona en promedio, y por lo tanto la enfermedad se propagará en la población.

- Si $\mathcal{R}_0 < 1$, cada persona infectada transmitirá la enfermedad a menos de una persona en promedio, lo que significa que eventualmente la epidemia se extinguirá en la población.

Ahora, de manera general, si queremos implementar una estrategia de control cuyo objetivo sea disminuir la tasa de transmisión en la población, podemos añadir un *término de control* al modelo (1) de la siguiente forma:

$$
    \begin{split}
            \frac{dS}{dt} &= -(1-u) \beta S I \\
            \frac{dI}{dt} &= (1-u) \beta S I  - \gamma I \\
            \frac{dR}{dt} &= \gamma I \\
        \end{split} \tag{2}
$$

donde $u$ es la *estrategia de control* aplicada sobre el sistema. Este parámetro cumple con la condición:

$$
    0 \leq u(t) \leq 1,  \quad \forall t \in [t_0, t_f] \tag{3}
$$

Veamos qué pasa si consideramos los extremos para este parámetro.

- Si $u=0$, tenemos *control nulo*, que implica no intervenir para detener la propagación de la enfermedad, y por lo tanto, la dinámica del sistema no se ve afectada.

- Si $u=1$, hay un *control máximo*, que corresponde a una intervención que detiene por completo la propagación de la enfermedad.

Como mencionamos anteriormente, es deseable para las autoridades de salud y gubernamentales que dicho control sea óptimo, al minimizar el costo que toma implementarlos, logrando reducir a lo más posible la propagación de la enfermedad. Veamos la manera en la que podemos formular este problema.

### Control óptimo

Consideremos la siguiente *función de costo*:

$$
    J(S,I,t) = h(I(t_f, t_f)) + \int_{t_0}^{t_f} g(I(\tau), u(\tau), \tau) \, d \tau \tag{4}
$$

donde $h(I(t_f, t_f))$ es el costo final, que en este contexto puede representar el costo asociado al estado final del sistema, por ejemplo, los costos de atención médica para las personas infectadas al tiempo $t_f$; el segundo término es el costo instantáneo a lo largo del periodo $[t_0,t_f]$. El objetivo es encontrar la *estrategia de control óptimo* $u^*$ que minimice a la función $J$.

Para resolver este tipo de problemas se puede utilizar la [*ecuación de Hamilton-Jacobi-Bellman*](https://en.wikipedia.org/wiki/Hamilton%E2%80%93Jacobi%E2%80%93Bellman_equation), la cual otorga una condición necesaria y suficiente para la optimalidad de un control con respecto a una función de costo. 

La expresión general de la ecuación de Hamilton-Jacobi-Bellman (HJB) es la siguiente:

$$
    \frac{\partial J}{\partial t} + \min_{u} \Big\{ g(x(t),u(t)) + \nabla J(x(t),t) \cdot f(x(t),u(t))^T \Big\} = 0 \tag{5}
$$

donde $x(t)$ representa al estado del sistema, en nuestro caso, el número de susceptibles e infectados, $f(x,u)$ es la dinámica del sistema, y $g(x,u)$ representa el costo instantáneo de tener un estado del sistema y aplicar las medidas de control $u$. 

Podemos encontrar la ley de control óptimo utilizando el [*principio de optimalidad de Bellman*](https://en.wikipedia.org/wiki/Bellman_equation#Bellman's_principle_of_optimality), el cual establece que una política óptima tiene la propiedad de que, sin importar el estado en el que se encuentre el sistema, el control que se elija a partir de ese punto debe ser óptima respecto a los estados restantes. En la práctica, equivale a aplicar el operador de minimización en (5), derivando respecto a $u$, y suponiendo que la $u$ que satisface la igualdad es la política de control que buscamos, es decir $u = u^*$. 

Para nuestro problema particular, es necesario proponer una función de costo instantáneo. Sea:

$$
    g(I,u,t) = a I^2 + b u^2 \quad a,b \in \mathbb{R} \tag{6}
$$

donde:

- $a I^2$ supone que el costo aumenta cuadráticamente con $I$, e intenta minimizar la cantidad de individuos infectados en el sistema.

- $b u^2$ representa el costo asociado con la implementación de la estrategia de control y análogo al término anterior, intenta minimizar la intensidad de la intervención.


Entonces, considerando el sistema (2) y la función (6), al sustituir en la ecuación de HJB obtenemos:

$$
    \frac{\partial J}{\partial t} + \min_{u \in \mathcal{U}} \Big\{ aI^2 + b u^2 - (1-u) \beta SI \frac{\partial J}{\partial S} + \left( (1-u) \beta SI - \gamma I \right) \frac{\partial J}{\partial I} \Big\} = 0 \tag{7}
$$

Aplicando el principio de optimalidad de Bellman, podemos encontrar la expresión para $u^*$:

$$
    \boxed{
    u^* = - \frac{\beta S I}{2b} \left( \frac{\partial J}{\partial S} - \frac{\partial J}{\partial I}\right)} \tag{8}
$$

Al sustituir $u = u^*$ en (7) obtenemos la ecuación de HJB para un modelo epidemiológico SIR con control en la tasa de transmisión:

$$
    \boxed{
    \frac{\partial J}{\partial t} + a I^2 - \frac{(\beta SI)^2}{2b} \left(\frac12 \left[ \left( \frac{\partial J}{\partial S} \right)^2 + \left( \frac{\partial J}{\partial I} \right)^2 \right] - \frac{\partial J}{\partial S} \frac{\partial J}{\partial I} \right) + \beta SI \left( \frac{\partial J}{\partial I} - \frac{\partial J}{\partial S} \right) - \gamma I \frac{\partial J}{\partial I} = 0 
    } \tag{9}
$$

Notemos que se trata de una ecuación diferencial parcial no lineal de primer orden. En general es complicado obtener una solución analítica en forma cerrada para este tipo de ecuaciones, así que debemos recurrir a métodos numéricos para conseguir una solución aproximada. Los métodos tradicionales para la resolución de ecuaciones diferenciales parciales son el método de diferencias finitas, el método del elemento finito, métodos de transformadas de Fourier, etcétera. Sin embargo, para el desarrollo de este trabajo se utilizaron redes neuronales artificiales.

### Redes neuronales artificiales informadas por la física

Las [*redes neuronales artificiales informadas por la física*](https://www.sciencedirect.com/science/article/abs/pii/S0021999118307125) en inglés *Physics Informed Neural Networks* (*PINN*), son una extensión reciente del concepto de las redes neuronales artificiales, [*Artificial Neural Networks*](https://www.ibm.com/topics/neural-networks) (*ANN*). Al igual que las ANN, las PINN están diseñadas para aprender relaciones complejas a partir de datos de entrenamiento. Sin embargo, las PINN incorporan el conocimiento previo sobre el sistema físico que se modela, a través de ecuaciones diferenciales que describen la evolución temporal de las variables de interés.

Durante su entrenamiento, las PINN buscan minimizar la diferencia entre las predicciones de la red y la ecuación o ecuaciones diferenciales gobernantes del sistema. De manera muy general, el algoritmo detrás de una PINN se puede resumir como sigue:

- Entrenamos una red neuronal para aproximar la solución de una ecuación diferencial de la forma $\frac{dy}{dt} = f(y,t)$. La función toma como entrada a $t$ y produce una salida $\hat{y}$ que es una aproximación de $y$.

- Como las redes neuronales son diferenciables, se calcula  $\hat{y}'$ con respecto a $t$ utilizando el algoritmo de [*propagación hacia atrás*](http://personal.cimat.mx:8181/~mrivera/cursos/aprendizaje_profundo/backprop/backpropagation.html#redes-multicapa-y-el-algoritmo-de-backpropagation).

- Finalmente, se define una *función de pérdida* (*loss function*) que mide cuán bien se satisface la ecuación diferencial. Suele usarse el error cuadrático médio (MSE) como función de pérdida de la red: 
    $$
        L = \frac{1}{N} \sum_{i=1}^N \left( \frac{d \hat{y_i}}{dt} - f(y_i, t_i) \right)^2 \tag{10}
    $$

De manera gráfica, se muestra el algoritmo de una PINN que resuelva la ecuación de HJB (9):

<div style="text-align: center">
    <figure>
        <img src='img\pinn_diagram.png' width='800' height='500' />
        <figcaption> Esquema del algoritmo de una PINN.</figcaption>
    </figure>
</div>

La idea es utilizar la PINN para aproximar la solución para (9), y una vez que obtengamos dicha solución, obtener a $u^*$ de acuerdo a la forma (8).